<a href="https://colab.research.google.com/github/Roque7/github-slideshow/blob/main/TEST1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Modelo clasificación de texto en español con dataset Amazon Reviews**

**Nota para el profesor:** He utilizado un dataset en español de reseñas de Amazon. Mi principal problema ha sido el tamaño del dataset, pues me daba errores al ser tan grande (200000). Para solucionar el problema he decidido utlizar el split de validación (5000), lo he revuelto y he creado el split de training de 4000 y el de test de 1000.

In [1]:
pip install datasets evaluate transformers[sentencepiece] -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 7.6 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

ModuleNotFoundError: No module named 'huggingface_hub'

In [ ]:
from datasets import load_dataset
raw_dataset = load_dataset('amazon_reviews_multi', 'es')

ModuleNotFoundError: No module named 'datasets'

In [ ]:
from datasets import load_dataset
raw_dataset = load_dataset('amazon_reviews_multi','es')

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
})

In [ ]:
raw_dataset["validation"]

Dataset({
    features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
    num_rows: 5000
})

In [ ]:
from datasets import DatasetDict,Dataset
new_dataset = raw_dataset["validation"].shuffle()
len_dataset = len(raw_dataset["validation"])
train_dataset = new_dataset[0:int(len_dataset*0.8)]
test_dataset = new_dataset[int(len_dataset*0.8):]
final_dataset = DatasetDict({"train":Dataset.from_dict(train_dataset),"test":Dataset.from_dict(test_dataset)})

In [ ]:
final_dataset

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 1000
    })
})

In [ ]:
final_dataset["train"].to_pandas()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0554577,product_es_0364246,reviewer_es_0968720,1,"NO FUNCIONA PARA seat toledo 3, quitarlo en op...",Mal! no funciona bien,es,automotive
1,es_0528950,product_es_0081260,reviewer_es_0743778,3,Los numeros 7 y 0 estaban practicamente borrados,Barato,es,sports
2,es_0631906,product_es_0828082,reviewer_es_0056946,2,Esto muy descontento con el tiempo me he dado ...,No es lo que parece faltan resistencia,es,lawn_and_garden
3,es_0265619,product_es_0489796,reviewer_es_0662659,4,Mucho más de lo que había pensado y mucho más ...,La cartera perfecta,es,luggage
4,es_0286766,product_es_0706218,reviewer_es_0891818,4,Bueno aunque llego con una parte abierta,Necesario,es,furniture
...,...,...,...,...,...,...,...,...
3995,es_0903809,product_es_0300909,reviewer_es_0234163,4,"La arena está muy bien, ya la he comprado vari...","SEUR no envía a la dirección marcada, sino a u...",es,pet_products
3996,es_0305447,product_es_0868372,reviewer_es_0265306,4,Cumple con la función que promete. Es sólido.,Es sólido,es,furniture
3997,es_0740744,product_es_0966005,reviewer_es_0475744,2,"El olor que deja es bueno, pero pensaba que de...",No deja el pelo sedoso,es,beauty
3998,es_0262940,product_es_0375347,reviewer_es_0561509,3,Dentro de las expectativas,Aceptable,es,kitchen


In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding

model_checkpoint = "mrm8488/electricidad-base-discriminator"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
def tokenize_function(example):
    return tokenizer(example["review_title"], truncation=True)

In [ ]:
tokenized_dataset = final_dataset.map(tokenize_function, batched=True)
tokenized_dataset

  0%|          | 0/4 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [ ]:
tokenized_dataset = tokenized_dataset.rename_column('stars','label')
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'label', 'review_body', 'review_title', 'language', 'product_category', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'label', 'review_body', 'review_title', 'language', 'product_category', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments("clasificador-amazonreviews",evaluation_strategy="epoch")

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=6)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--mrm8488--electricidad-base-discriminator/snapshots/1353d86e74c5ae322590dcda6e216259b1f72b67/config.json
Model config ElectraConfig {
  "_name_or_path": "mrm8488/electricidad-base-discriminator",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,


In [ ]:
import evaluate
import numpy as np

def compute_metrics(eval_preds):
  metric = evaluate.load("accuracy")
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: product_category, review_body, review_id, language, reviewer_id, review_title, product_id. If product_category, review_body, review_id, language, reviewer_id, review_title, product_id are not expected by `ElectraForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 4000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1500
  Number of trainable parameters = 109855494


Epoch,Training Loss,Validation Loss,Accuracy
1,1.442100,1.301417,0.392000
2,1.213200,1.299452,0.418000
3,1.069200,1.332963,0.441000


Saving model checkpoint to clasificador-amazonreviews/checkpoint-500
Configuration saved in clasificador-amazonreviews/checkpoint-500/config.json
Model weights saved in clasificador-amazonreviews/checkpoint-500/pytorch_model.bin
tokenizer config file saved in clasificador-amazonreviews/checkpoint-500/tokenizer_config.json
Special tokens file saved in clasificador-amazonreviews/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: product_category, review_body, review_id, language, reviewer_id, review_title, product_id. If product_category, review_body, review_id, language, reviewer_id, review_title, product_id are not expected by `ElectraForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


Saving model checkpoint to clasificador-amazonreviews/checkpoint-1000
Configuration saved in clasificador-amazonreviews/checkpoint-1000/config.json
Model weights saved in clasificador-amazonreviews/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in clasificador-amazonreviews/checkpoint-1000/tokenizer_config.json
Special tokens file saved in clasificador-amazonreviews/checkpoint-1000/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: product_category, review_body, review_id, language, reviewer_id, review_title, product_id. If product_category, review_body, review_id, language, reviewer_id, review_title, product_id are not expected by `ElectraForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoint to clasificador-amazonreviews/checkpoint-1500
Co

TrainOutput(global_step=1500, training_loss=1.241508544921875, metrics={'train_runtime': 2931.1165, 'train_samples_per_second': 4.094, 'train_steps_per_second': 0.512, 'total_flos': 72099129581856.0, 'train_loss': 1.241508544921875, 'epoch': 3.0})

In [ ]:
%cd clasificador-amazonreviews
trainer.push_to_hub(commit_message="Training complete", tags="classification")

Saving model checkpoint to clasificador-amazonreviews
Configuration saved in clasificador-amazonreviews/config.json


/content/clasificador-amazonreviews/clasificador-amazonreviews


Model weights saved in clasificador-amazonreviews/pytorch_model.bin
tokenizer config file saved in clasificador-amazonreviews/tokenizer_config.json
Special tokens file saved in clasificador-amazonreviews/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file clasificador-amazonreviews/pytorch_model.bin:   0%|          | 32.0k/419M [00:00<?, ?B/s]

remote: Scanning LFS files for validity...        
remote: LFS file scan complete.        
To https://huggingface.co/esoria3/clasificador-amazonreviews
   cdb2290..988051c  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/esoria3/clasificador-amazonreviews
   cdb2290..988051c  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'Accuracy', 'type': 'accuracy', 'value': 0.441}]}
To https://huggingface.co/esoria3/clasificador-amazonreviews
   988051c..6b7ffd0  main -> main

   988051c..6b7ffd0  main -> main



'https://huggingface.co/esoria3/clasificador-amazonreviews/commit/988051c41b5ed5e996f5cf0838cb957bc23c0b74'

In [ ]:
from transformers import pipeline
classifier = pipeline('text-classification', model='esoria3/clasificador-amazonreviews')

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--esoria3--clasificador-amazonreviews/snapshots/6b7ffd0de53c86036fce503b481c2f6e5b060f85/config.json
Model config ElectraConfig {
  "_name_or_path": "esoria3/clasificador-amazonreviews",
  "architectures": [
    "ElectraForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--esoria3--clasificador-amazonreviews/snapshots/6b7ffd0de53c86036fce503b481c2f6e5b060f85/pytorch_model.bin
All model checkpoint weights were used when initializing ElectraForSequenceClassification.

All the weights of ElectraForSequenceClassification were initialized from the model checkpoint at esoria3/clasificador-amazonreviews.
If your task is similar to the task the model of the checkpoint was trained on, you can already use ElectraForSequenceClassification for predictions without further training.


loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--esoria3--clasificador-amazonreviews/snapshots/6b7ffd0de53c86036fce503b481c2f6e5b060f85/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--esoria3--clasificador-amazonreviews/snapshots/6b7ffd0de53c86036fce503b481c2f6e5b060f85/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--esoria3--clasificador-amazonreviews/snapshots/6b7ffd0de53c86036fce503b481c2f6e5b060f85/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--esoria3--clasificador-amazonreviews/snapshots/6b7ffd0de53c86036fce503b481c2f6e5b060f85/tokenizer_config.json


In [ ]:
classifier("Funciona de maravilla.")

[{'label': 'LABEL_5', 'score': 0.5923536419868469}]

In [ ]:
classifier("Funciona fatal.")

[{'label': 'LABEL_1', 'score': 0.8219757080078125}]

In [ ]:
classifier("No estoy del todo contenta.")

[{'label': 'LABEL_2', 'score': 0.5347573161125183}]

In [ ]:
classifier("He perdido mi dinero.")

[{'label': 'LABEL_1', 'score': 0.8026193380355835}]

In [ ]:
classifier("Lo volveré a comprar.")

[{'label': 'LABEL_3', 'score': 0.4198615849018097}]